In [1]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DecimalType,IntegerType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialize Spark with Snowflake packages
spark = SparkSession.builder \
    .appName("SnowflakeLoad_LoanProduct") \
    .config("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.13.22,net.snowflake:spark-snowflake_2.12:2.11.0-spark_3.3") \
    .getOrCreate()

# my addition     
custom_schema = StructType([
    StructField("BORROWER_KEY_PK_SK", DecimalType(38, 0), True),  # Autoincrement PK
    StructField("ANNUAL_INCOME", StringType(), True),
    StructField("EMPLOYMENT_TITLE", StringType(), True),
    StructField("EMPLOYMENT_LENGTH", StringType(), True),
    StructField("HOMEOWNERSHIP", StringType(), True),
    StructField("EARLIEST_CREDIT_LINE_DATE", StringType(), True),
    StructField("VERIFICATION_STATUS", StringType(), True),
    StructField("ZIP_CODE", StringType(), True),
    StructField("ADDRESS_STATE", StringType(), True),
    StructField("STATE", StringType(), True),
    StructField("DELINQ_2YEARS",StringType(), True),
    StructField("AVG_FICO_RANGE",StringType(), True),
    StructField("OPEN_ACCOUNT", StringType(), True),
    StructField("PUBLIC_RECORD", StringType(), True),
    StructField("TOTAL_ACCOUNT", StringType(), True),
    StructField("LAST_PAYMENT_DAY", StringType(), True),
    StructField("LAST_PAYMENT_AMOUNT", StringType(), True),
    StructField("NEXT_PAYMENT_DAY", StringType(), True),
    StructField("MTHS_SINCE_LAST_MAJOR_DEROG", StringType(), True),
    StructField("ACTIVE_LOANS_LAST_24MONTH", StringType(), True),
    StructField("MORTGAGE_ACCOUNT", StringType(), True),
    StructField("HARDSHIP_FLAG", StringType(), True)
])
# Load CSV file into DataFrame (Replace 'your_file.csv' with actual filename)
csv_path = "Transformed_2014_18_v2.csv"
df = spark.read.option("header", True).csv(csv_path)  #.sample(withReplacement=False, fraction=0.0001, seed=42)


#### customize the csv file for Dim_borrower
columns_of_borrowers = [
    "annual_inc", "emp_title", "emp_length", "home_ownership","earliest_cr_line", "verification_status", "zip_code", "addr_state",
    "full_state_name","delinq_2yrs","avg_fico_range", "open_acc",
    "pub_rec", "total_acc",  
     
     "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d",
    "mths_since_last_major_derog", "open_il_24m", "mort_acc", "hardship_flag","id"
]
 
dim_borrower = df.select(columns_of_borrowers)
dim_borrower.show(5)

# Snowflake connection options
sf_options = {
    "sfURL": "https://WOA97553.east-us-2.azure.snowflakecomputing.com",
    "sfUser": 'hussien1',
    "sfPassword":'@Hussien123456',
    "sfDatabase": 'Loan_DB',
    "sfSchema": 'Loan_Schema',
    "sfWarehouse": 'loan_Warehouse',
    "autopushdown": "on",
    "usestagingtable": "on"
}

+----------+--------------------+----------+--------------+----------------+-------------------+--------+----------+---------------+-----------+--------------+--------+-------+---------+------------+---------------+------------+---------------------------+-----------+--------+-------------+
|annual_inc|           emp_title|emp_length|home_ownership|earliest_cr_line|verification_status|zip_code|addr_state|full_state_name|delinq_2yrs|avg_fico_range|open_acc|pub_rec|total_acc|last_pymnt_d|last_pymnt_amnt|next_pymnt_d|mths_since_last_major_derog|open_il_24m|mort_acc|hardship_flag|
+----------+--------------------+----------+--------------+----------------+-------------------+--------+----------+---------------+-----------+--------------+--------+-------+---------+------------+---------------+------------+---------------------------+-----------+--------+-------------+
|   65000.0|         Maintenance|       +10|      MORTGAGE|        Mar-2002|       Not Verified|   686xx|        NE|       N

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Column Mapping: Rename source columns to match DIM_BORROWER destination columns
column_mapping = {
    "annual_inc": "ANNUAL_INCOME",
    "emp_title": "EMPLOYMENT_TITLE",
    "emp_length": "EMPLOYMENT_LENGTH",
    "home_ownership": "HOMEOWNERSHIP",
    "earliest_cr_line": "EARLIEST_CREDIT_LINE_DATE",
    "verification_status": "VERIFICATION_STATUS",
    "zip_code": "ZIP_CODE",
    "addr_state": "ADDRESS_STATE",
    "full_state_name": "STATE",
    "delinq_2yrs": "DELINQ_2YEARS",
    "avg_fico_range": "AVG_FICO_RANGE",
    "open_acc": "OPEN_ACCOUNT",   # If open_acc is duplicated, keep it only once
    "pub_rec": "PUBLIC_RECORD",   # If pub_rec is duplicated, keep it only once
    "total_acc": "TOTAL_ACCOUNT",
    "last_pymnt_d": "LAST_PAYMENT_DAY",
    "last_pymnt_amnt": "LAST_PAYMENT_AMOUNT",
    "next_pymnt_d": "NEXT_PAYMENT_DAY",
    "mths_since_last_major_derog": "MTHS_SINCE_LAST_MAJOR_DEROG",
    "open_il_24m": "ACTIVE_LOANS_LAST_24MONTH",
    "mort_acc": "MORTGAGE_ACCOUNT",
    "hardship_flag": "HARDSHIP_FLAG",
    "id":"LOANPRODUCT_BK"
}

# Rename columns in the DataFrame according to the column mapping
for source_col, dest_col in column_mapping.items():
    if source_col in dim_borrower.columns:
        dim_borrower = dim_borrower.withColumnRenamed(source_col, dest_col)

# Define the window spec for row numbering
windowSpec = Window.orderBy(F.lit(1))  # Constant value to ensure numbering starts from 1

# Add the BORROWER_KEY_PK_SK column with row numbers starting from 1
dim_borrower = dim_borrower.withColumn("BORROWER_KEY_PK_SK", F.row_number().over(windowSpec))

# List of columns to select (ensure that the column names match after renaming)
columns_of_borrowers = [
    "BORROWER_KEY_PK_SK", "ANNUAL_INCOME", "EMPLOYMENT_TITLE", "EMPLOYMENT_LENGTH", "HOMEOWNERSHIP", "EARLIEST_CREDIT_LINE_DATE", "VERIFICATION_STATUS", 
    "ZIP_CODE", "ADDRESS_STATE", "STATE", "DELINQ_2YEARS", "AVG_FICO_RANGE", "OPEN_ACCOUNT", "PUBLIC_RECORD", "TOTAL_ACCOUNT", 
    "LAST_PAYMENT_DAY", "LAST_PAYMENT_AMOUNT", "NEXT_PAYMENT_DAY", "MTHS_SINCE_LAST_MAJOR_DEROG", "ACTIVE_LOANS_LAST_24MONTH", "MORTGAGE_ACCOUNT", 
    "HARDSHIP_FLAG","LOANPRODUCT_BK"
]

# Select the required columns
dim_borrower = dim_borrower.select(columns_of_borrowers)

# Show DataFrame to verify changes
print("Transformed Data for DIM_BORROWER:")
dim_borrower.show()


Transformed Data for DIM_BORROWER:
+------------------+-------------+--------------------+-----------------+-------------+-------------------------+-------------------+--------+-------------+------------+-------------+--------------+------------+-------------+-------------+----------------+-------------------+----------------+---------------------------+-------------------------+----------------+-------------+
|BORROWER_KEY_PK_SK|ANNUAL_INCOME|    EMPLOYMENT_TITLE|EMPLOYMENT_LENGTH|HOMEOWNERSHIP|EARLIEST_CREDIT_LINE_DATE|VERIFICATION_STATUS|ZIP_CODE|ADDRESS_STATE|       STATE|DELINQ_2YEARS|AVG_FICO_RANGE|OPEN_ACCOUNT|PUBLIC_RECORD|TOTAL_ACCOUNT|LAST_PAYMENT_DAY|LAST_PAYMENT_AMOUNT|NEXT_PAYMENT_DAY|MTHS_SINCE_LAST_MAJOR_DEROG|ACTIVE_LOANS_LAST_24MONTH|MORTGAGE_ACCOUNT|HARDSHIP_FLAG|
+------------------+-------------+--------------------+-----------------+-------------+-------------------------+-------------------+--------+-------------+------------+-------------+--------------+---------

In [3]:
dim_borrower.printSchema()

root
 |-- BORROWER_KEY_PK_SK: integer (nullable = false)
 |-- ANNUAL_INCOME: string (nullable = true)
 |-- EMPLOYMENT_TITLE: string (nullable = true)
 |-- EMPLOYMENT_LENGTH: string (nullable = true)
 |-- HOMEOWNERSHIP: string (nullable = true)
 |-- EARLIEST_CREDIT_LINE_DATE: string (nullable = true)
 |-- VERIFICATION_STATUS: string (nullable = true)
 |-- ZIP_CODE: string (nullable = true)
 |-- ADDRESS_STATE: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- DELINQ_2YEARS: string (nullable = true)
 |-- AVG_FICO_RANGE: string (nullable = true)
 |-- OPEN_ACCOUNT: string (nullable = true)
 |-- PUBLIC_RECORD: string (nullable = true)
 |-- TOTAL_ACCOUNT: string (nullable = true)
 |-- LAST_PAYMENT_DAY: string (nullable = true)
 |-- LAST_PAYMENT_AMOUNT: string (nullable = true)
 |-- NEXT_PAYMENT_DAY: string (nullable = true)
 |-- MTHS_SINCE_LAST_MAJOR_DEROG: string (nullable = true)
 |-- ACTIVE_LOANS_LAST_24MONTH: string (nullable = true)
 |-- MORTGAGE_ACCOUNT: string (nullabl

In [4]:
dim_borrower.show(3)

+------------------+-------------+-------------------+-----------------+-------------+-------------------------+-------------------+--------+-------------+--------+-------------+--------------+------------+-------------+-------------+----------------+-------------------+----------------+---------------------------+-------------------------+----------------+-------------+
|BORROWER_KEY_PK_SK|ANNUAL_INCOME|   EMPLOYMENT_TITLE|EMPLOYMENT_LENGTH|HOMEOWNERSHIP|EARLIEST_CREDIT_LINE_DATE|VERIFICATION_STATUS|ZIP_CODE|ADDRESS_STATE|   STATE|DELINQ_2YEARS|AVG_FICO_RANGE|OPEN_ACCOUNT|PUBLIC_RECORD|TOTAL_ACCOUNT|LAST_PAYMENT_DAY|LAST_PAYMENT_AMOUNT|NEXT_PAYMENT_DAY|MTHS_SINCE_LAST_MAJOR_DEROG|ACTIVE_LOANS_LAST_24MONTH|MORTGAGE_ACCOUNT|HARDSHIP_FLAG|
+------------------+-------------+-------------------+-----------------+-------------+-------------------------+-------------------+--------+-------------+--------+-------------+--------------+------------+-------------+-------------+----------------+-

In [5]:
# Assuming df is your DataFrame
sampled_df = dim_borrower.sample(fraction=0.0001, seed=42) 
sampled_df.show(3)

+------------------+-------------+--------------------+-----------------+-------------+-------------------------+-------------------+--------+-------------+--------+-------------+--------------+------------+-------------+-------------+----------------+-------------------+----------------+---------------------------+-------------------------+----------------+-------------+
|BORROWER_KEY_PK_SK|ANNUAL_INCOME|    EMPLOYMENT_TITLE|EMPLOYMENT_LENGTH|HOMEOWNERSHIP|EARLIEST_CREDIT_LINE_DATE|VERIFICATION_STATUS|ZIP_CODE|ADDRESS_STATE|   STATE|DELINQ_2YEARS|AVG_FICO_RANGE|OPEN_ACCOUNT|PUBLIC_RECORD|TOTAL_ACCOUNT|LAST_PAYMENT_DAY|LAST_PAYMENT_AMOUNT|NEXT_PAYMENT_DAY|MTHS_SINCE_LAST_MAJOR_DEROG|ACTIVE_LOANS_LAST_24MONTH|MORTGAGE_ACCOUNT|HARDSHIP_FLAG|
+------------------+-------------+--------------------+-----------------+-------------+-------------------------+-------------------+--------+-------------+--------+-------------+--------------+------------+-------------+-------------+---------------

In [6]:
sampled_df.printSchema()

root
 |-- BORROWER_KEY_PK_SK: integer (nullable = false)
 |-- ANNUAL_INCOME: string (nullable = true)
 |-- EMPLOYMENT_TITLE: string (nullable = true)
 |-- EMPLOYMENT_LENGTH: string (nullable = true)
 |-- HOMEOWNERSHIP: string (nullable = true)
 |-- EARLIEST_CREDIT_LINE_DATE: string (nullable = true)
 |-- VERIFICATION_STATUS: string (nullable = true)
 |-- ZIP_CODE: string (nullable = true)
 |-- ADDRESS_STATE: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- DELINQ_2YEARS: string (nullable = true)
 |-- AVG_FICO_RANGE: string (nullable = true)
 |-- OPEN_ACCOUNT: string (nullable = true)
 |-- PUBLIC_RECORD: string (nullable = true)
 |-- TOTAL_ACCOUNT: string (nullable = true)
 |-- LAST_PAYMENT_DAY: string (nullable = true)
 |-- LAST_PAYMENT_AMOUNT: string (nullable = true)
 |-- NEXT_PAYMENT_DAY: string (nullable = true)
 |-- MTHS_SINCE_LAST_MAJOR_DEROG: string (nullable = true)
 |-- ACTIVE_LOANS_LAST_24MONTH: string (nullable = true)
 |-- MORTGAGE_ACCOUNT: string (nullabl

In [8]:

try:
    print("Testing Snowflake connection...")
    test_df = spark.read \
        .format("net.snowflake.spark.snowflake") \
        .options(**sf_options) \
        .option("query", "SELECT 1 AS test_value") \
        .load()
    test_df.show()
    print("Connection test successful!")
 
    print("Writing data to Snowflake...")
    dim_borrower.write \
        .format("snowflake") \
        .options(**sf_options) \
        .option("dbtable", "DIM_BORROWER") \
        .mode("append") \
        .save()
 
    print("Data load complete!")
 #######################

#############################################
    print("Reading data back from Snowflake to verify...")
    snowflake_df = spark.read \
        .format("net.snowflake.spark.snowflake") \
        .options(**sf_options) \
        .option("query", "SELECT * FROM Loan_DB.Loan_Schema.DIM_BORROWER") \
        .load()
 
    snowflake_df.show()
 
except Exception as e:
    print("Error occurred:", str(e))
 
#finally:
    #spark.stop()

Testing Snowflake connection...
+----------+
|TEST_VALUE|
+----------+
|         1|
+----------+

Connection test successful!
Writing data to Snowflake...
Data load complete!
Reading data back from Snowflake to verify...
+------------------+-------------+--------------------+-----------------+-------------+-------------------------+-------------------+--------+-------------+--------------+-------------+--------------+------------+-------------+-------------+----------------+-------------------+----------------+---------------------------+-------------------------+----------------+-------------+
|BORROWER_KEY_PK_SK|ANNUAL_INCOME|    EMPLOYMENT_TITLE|EMPLOYMENT_LENGTH|HOMEOWNERSHIP|EARLIEST_CREDIT_LINE_DATE|VERIFICATION_STATUS|ZIP_CODE|ADDRESS_STATE|         STATE|DELINQ_2YEARS|AVG_FICO_RANGE|OPEN_ACCOUNT|PUBLIC_RECORD|TOTAL_ACCOUNT|LAST_PAYMENT_DAY|LAST_PAYMENT_AMOUNT|NEXT_PAYMENT_DAY|MTHS_SINCE_LAST_MAJOR_DEROG|ACTIVE_LOANS_LAST_24MONTH|MORTGAGE_ACCOUNT|HARDSHIP_FLAG|
+----------------

# Load Dim_Borrower at onne cell

In [2]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DecimalType,IntegerType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialize Spark with Snowflake packages
spark = SparkSession.builder \
    .appName("SnowflakeLoad_LoanProduct") \
    .config("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.13.22,net.snowflake:spark-snowflake_2.12:2.11.0-spark_3.3") \
    .getOrCreate()

# my addition     
custom_schema = StructType([
    StructField("BORROWER_KEY_PK_SK", DecimalType(38, 0), True),  # Autoincrement PK
    StructField("ANNUAL_INCOME", StringType(), True),
    StructField("EMPLOYMENT_TITLE", StringType(), True),
    StructField("EMPLOYMENT_LENGTH", StringType(), True),
    StructField("HOMEOWNERSHIP", StringType(), True),
    StructField("EARLIEST_CREDIT_LINE_DATE", StringType(), True),
    StructField("VERIFICATION_STATUS", StringType(), True),
    StructField("ZIP_CODE", StringType(), True),
    StructField("ADDRESS_STATE", StringType(), True),
    StructField("STATE", StringType(), True),
    StructField("DELINQ_2YEARS",StringType(), True),
    StructField("AVG_FICO_RANGE",StringType(), True),
    StructField("OPEN_ACCOUNT", StringType(), True),
    StructField("PUBLIC_RECORD", StringType(), True),
    StructField("TOTAL_ACCOUNT", StringType(), True),
    StructField("LAST_PAYMENT_DAY", StringType(), True),
    StructField("LAST_PAYMENT_AMOUNT", StringType(), True),
    StructField("NEXT_PAYMENT_DAY", StringType(), True),
    StructField("MTHS_SINCE_LAST_MAJOR_DEROG", StringType(), True),
    StructField("ACTIVE_LOANS_LAST_24MONTH", StringType(), True),
    StructField("MORTGAGE_ACCOUNT", StringType(), True),
    StructField("HARDSHIP_FLAG", StringType(), True)
])
# Load CSV file into DataFrame (Replace 'your_file.csv' with actual filename)
csv_path = "Transformed_2014_18_v2.csv"
df = spark.read.option("header", True).csv(csv_path)  #.sample(withReplacement=False, fraction=0.0001, seed=42)


#### customize the csv file for Dim_borrower
columns_of_borrowers = [
    "annual_inc", "emp_title", "emp_length", "home_ownership","earliest_cr_line", "verification_status", "zip_code", "addr_state",
    "full_state_name","delinq_2yrs","avg_fico_range", "open_acc",
    "pub_rec", "total_acc",  
     
     "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d",
    "mths_since_last_major_derog", "open_il_24m", "mort_acc", "hardship_flag","id"
]
 
dim_borrower = df.select(columns_of_borrowers)
dim_borrower.show(5)

# Snowflake connection options
sf_options = {
    "sfURL": "https://WOA97553.east-us-2.azure.snowflakecomputing.com",
    "sfUser": 'hussien1',
    "sfPassword":'@Hussien123456',
    "sfDatabase": 'Loan_DB',
    "sfSchema": 'Loan_Schema',
    "sfWarehouse": 'loan_Warehouse',
    "autopushdown": "on",
    "usestagingtable": "on"
}


from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Column Mapping: Rename source columns to match DIM_BORROWER destination columns
column_mapping = {
    "annual_inc": "ANNUAL_INCOME",
    "emp_title": "EMPLOYMENT_TITLE",
    "emp_length": "EMPLOYMENT_LENGTH",
    "home_ownership": "HOMEOWNERSHIP",
    "earliest_cr_line": "EARLIEST_CREDIT_LINE_DATE",
    "verification_status": "VERIFICATION_STATUS",
    "zip_code": "ZIP_CODE",
    "addr_state": "ADDRESS_STATE",
    "full_state_name": "STATE",
    "delinq_2yrs": "DELINQ_2YEARS",
    "avg_fico_range": "AVG_FICO_RANGE",
    "open_acc": "OPEN_ACCOUNT",   # If open_acc is duplicated, keep it only once
    "pub_rec": "PUBLIC_RECORD",   # If pub_rec is duplicated, keep it only once
    "total_acc": "TOTAL_ACCOUNT",
    "last_pymnt_d": "LAST_PAYMENT_DAY",
    "last_pymnt_amnt": "LAST_PAYMENT_AMOUNT",
    "next_pymnt_d": "NEXT_PAYMENT_DAY",
    "mths_since_last_major_derog": "MTHS_SINCE_LAST_MAJOR_DEROG",
    "open_il_24m": "ACTIVE_LOANS_LAST_24MONTH",
    "mort_acc": "MORTGAGE_ACCOUNT",
    "hardship_flag": "HARDSHIP_FLAG",
    "id":"LOANPRODUCT_BK"
}

# Rename columns in the DataFrame according to the column mapping
for source_col, dest_col in column_mapping.items():
    if source_col in dim_borrower.columns:
        dim_borrower = dim_borrower.withColumnRenamed(source_col, dest_col)

# Define the window spec for row numbering
windowSpec = Window.orderBy(F.lit(1))  # Constant value to ensure numbering starts from 1

# Add the BORROWER_KEY_PK_SK column with row numbers starting from 1
dim_borrower = dim_borrower.withColumn("BORROWER_KEY_PK_SK", F.row_number().over(windowSpec))

# List of columns to select (ensure that the column names match after renaming)
columns_of_borrowers = [
    "BORROWER_KEY_PK_SK", "ANNUAL_INCOME", "EMPLOYMENT_TITLE", "EMPLOYMENT_LENGTH", "HOMEOWNERSHIP", "EARLIEST_CREDIT_LINE_DATE", "VERIFICATION_STATUS", 
    "ZIP_CODE", "ADDRESS_STATE", "STATE", "DELINQ_2YEARS", "AVG_FICO_RANGE", "OPEN_ACCOUNT", "PUBLIC_RECORD", "TOTAL_ACCOUNT", 
    "LAST_PAYMENT_DAY", "LAST_PAYMENT_AMOUNT", "NEXT_PAYMENT_DAY", "MTHS_SINCE_LAST_MAJOR_DEROG", "ACTIVE_LOANS_LAST_24MONTH", "MORTGAGE_ACCOUNT", 
    "HARDSHIP_FLAG","LOANPRODUCT_BK"
]

# Select the required columns
dim_borrower = dim_borrower.select(columns_of_borrowers)

# Show DataFrame to verify changes
print("Transformed Data for DIM_BORROWER:")
dim_borrower.show(5)



#df_sampled = dim_borrower.sample(withReplacement=False, fraction=0.0001)
try:
    print("Testing Snowflake connection...")
    test_df = spark.read \
        .format("net.snowflake.spark.snowflake") \
        .options(**sf_options) \
        .option("query", "SELECT 1 AS test_value") \
        .load()
    test_df.show()
    print("Connection test successful!")
 
    print("Writing data to Snowflake...")
    dim_borrower.write \
        .format("snowflake") \
        .options(**sf_options) \
        .option("dbtable", "DIM_BORROWER") \
        .mode("append") \
        .save()
 
    print("Data load complete!")
 #######################

#############################################
    """
    print("Reading data back from Snowflake to verify...")
    snowflake_df = spark.read \
        .format("net.snowflake.spark.snowflake") \
        .options(**sf_options) \
        .option("query", "SELECT * FROM Loan_DB.Loan_Schema.DIM_BORROWER") \
        .load()
 
    snowflake_df.show()
 """
except Exception as e:
    print("Error occurred:", str(e))
 
#finally:
    #spark.stop()

+----------+--------------------+----------+--------------+----------------+-------------------+--------+----------+---------------+-----------+--------------+--------+-------+---------+------------+---------------+------------+---------------------------+-----------+--------+-------------+------+
|annual_inc|           emp_title|emp_length|home_ownership|earliest_cr_line|verification_status|zip_code|addr_state|full_state_name|delinq_2yrs|avg_fico_range|open_acc|pub_rec|total_acc|last_pymnt_d|last_pymnt_amnt|next_pymnt_d|mths_since_last_major_derog|open_il_24m|mort_acc|hardship_flag|    id|
+----------+--------------------+----------+--------------+----------------+-------------------+--------+----------+---------------+-----------+--------------+--------+-------+---------+------------+---------------+------------+---------------------------+-----------+--------+-------------+------+
|   65000.0|         Maintenance|       +10|      MORTGAGE|        Mar-2002|       Not Verified|   686x